In [17]:
import time, random
from collections import deque, namedtuple

# gym toolkit is a collection of environments 
# used to test reinforcement learning
import gym

import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam


_Technicall you would use PIL and pyvirtualdisplay to handle the graphics but they are bugging lmao_

### Hyperparameters

In [10]:
MEMORY_SIZE = 100_000
GAMMA = 0.995
ALPHA = 1e-3
NUM_STEPS_FOR_UPDATE = 4
TAU = 1e-3  
MINIBATCH_SIZE = 64
E_DECAY = 0.995
E_MIN = 0.01
SEED = 0

### Defining the Environment

##### Observation Space
The agent's observation space consists of a state vector with 8 variables:
* Its $(x,y)$ coordinates. The landing pad is always at coordinates $(0,0)$.
* Its linear velocities $(\dot x,\dot y)$.
* Its angle $\theta$.
* Its angular velocity $\dot \theta$.
* Two booleans, $l$ and $r$, that represent whether each leg is in contact with the ground or not.

##### Reward
The Lunar Lander environment has the following reward system:
* Landing on the landing pad and coming to rest is about 100-140 points.
* If the lander moves away from the landing pad, it loses reward. 
* If the lander crashes, it receives -100 points.
* If the lander comes to rest, it receives +100 points.
* Each leg with ground contact is +10 points.
* Firing the main engine is -0.3 points each frame.
* Firing the side engine is -0.03 points each frame.

##### Action Space
Do nothing = 0, Fire right engine = 1, main engine = 2, left = 3

##### Terminal State
An episode ends (i.e the environment enters a terminal state) if:

* The lunar lander crashes (i.e if the body of the lunar lander comes in contact with the surface of the moon).
* The lander's $x$-coordinate is greater than 1.

In [22]:
tf.random.set_seed(random.seed(SEED))

env = gym.make('LunarLander-v2')
env.reset()

# get info about environment: 
# size of state vector and number of valid actions
state_size = env.observation_space.shape
num_actions = env.action_space.n

print(f"State size: {state_size}")
print(f"Number of actions: {num_actions}")

State size: (8,)
Number of actions: 4


In [26]:
initial_state = env.reset()
action = 0

# Run a single time step of the environment's dynamics with the given action
next_state, reward, done, _, info = env.step(action)

with np.printoptions(formatter={'float': '{:.3f}'.format}):
    print("Initial State:", initial_state)
    print("Action:", action)
    print("Next State:", next_state)
    print("Reward Received:", reward)
    print("Episode Terminated:", done)
    print("Info:", info)

Initial State: (array([-0.008, 1.412, -0.803, 0.028, 0.009, 0.182, 0.000, 0.000],
      dtype=float32), {})
Action: 0
Next State: [-0.016 1.412 -0.802 0.002 0.018 0.180 0.000 0.000]
Reward Received: -0.754248141704295
Episode Terminated: False
Info: {}


### Deep Q-Network

In cases where both the state and action space are discrete we can estimate the action-value function iteratively by using the Bellman equation:

$$
Q_{i+1}(s,a) = R + \gamma \max_{a'}Q_i(s',a')
$$

This iterative method converges to the optimal action-value function $Q^*(s, a)$ as $i \to\infty$, which means if the agent gradually explore the state-action space and keep updating its estimate the Q function will converge to the optimal. However, for continous state-action space, you can't explore the entire space so you need to use a neural network to estimate the action-value function $Q(s,a) \approx Q^*(s,a)$. This is called a Q-network and it can be trained by adjusting weights at each iteration to minimize the MSE in Bellman Eq.

Below are two methods to avoid its instabilities:

#### 1. Target network 

The target values are given by: 
$$
y = R + \gamma \max_{a'}Q(s',a';w)
$$

We are adjusting the weights w to minimize the following error:
$$
\overbrace{\underbrace{R + \gamma \max_{a'}Q(s',a'; w)}_{\rm {y~target}} - Q(s,a;w)}^{\rm {Error}}
$$

The fact that y target is changing on every iteration is a problem. We need to create a separate neural network for generating y targets. It is called the **target $\hat Q$-Network** and will have the same architecture as the original Q network. The error is now:
$$
\overbrace{\underbrace{R + \gamma \max_{a'}\hat{Q}(s',a'; w^-)}_{\rm {y~target}} - Q(s,a;w)}^{\rm {Error}}
$$

* Every C time steps we use target network to generate y targets
* Then update weights of target Q network using weights of Q network
* We will use a soft update:
$$
w^-\leftarrow \tau w + (1 - \tau) w^-
$$

In this exercise you will create the $Q$ and target $\hat Q$ networks and set the optimizer. Remember that the Deep $Q$-Network (DQN) is a neural network that approximates the action-value function $Q(s,a)\approx Q^*(s,a)$. It does this by learning how to map states to $Q$ values.

In [32]:
q_network = Sequential([
    Input(state_size), 
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_actions, activation='linear')
])

target_q_network = Sequential([
    Input(state_size),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_actions, activation='linear')
])

optimizer = Adam(learning_rate=ALPHA)
target_q_network.set_weights(q_network.get_weights())

#### 2. Experience Replay

* s, a, reward the agent experiences are sequential by nature -- strong correlations = problem in learning
    * Experience replay generates uncorrelated experiences for training the agent
    * This avoid problematic correlations, oscillations, and instabilities
* Store the agent's experiences (s, a, reward receives) in a memory buffer 
* Then sample a random mini-batch of experiences form the buffer for learning

The experience tuples $(S_t, A_t, R_t, S_{t+1})$ will be added to the memory buffer at each time step as the agent interacts with the environment. We will use named tuples.

In [27]:
experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

# There is a good picture in Notion that summarizes the whole DQL algorithm with Experience Replay

### Deep Q-Learning Algorithm (with Experience Replay)

To compute the loss between the y targets and $Q(s,a)$ values, we will set the target euqal to:
$$
\begin{equation}
    y_j =
    \begin{cases}
      R_j & \text{if episode terminates at step  } j+1\\
      R_j + \gamma \max_{a'}\hat{Q}(s_{j+1},a') & \text{otherwise}\\
    \end{cases}       
\end{equation}
$$

The compute_loss function will take a mini-batch of experience tuples, unpacked to extract its elements which are tf Tensors with size = size of mini-batch. i.e. if mini-batch size = 64 then "rewards" extracted will be tf Tensor of 64 elements. The MSE loss is imported from keras but ofc you can implement it yourself.

In [38]:
# Line 12 of the algorithm

def compute_loss(experiences, gamma, q_network, target_q_network):
    
    # Unpack the mini-batch of experience tuple
    states, actions, rewards, next_states, dones = experiences

    # Compute max Q^(s, a) for the next states using the target network
    max_qsa = tf.reduce_max(target_q_network(next_states, training=False), axis = -1)

    # Set y (target values) = R if episode terminates, otherwise y = R + gamma * max Q^(s, a)
    y_targets = rewards + (gamma * max_qsa * (1 - dones))

    # Get the Q values for the current state
    q_values = q_network(states)

    # Get the Q values for the actions taken
    action_q_values = tf.gather_nd(q_values, tf.stack([tf.range(q_values.shape[0]), tf.cast(actions, tf.int32)], axis = 1))

    loss = MSE(y_targets, action_q_values)

    return loss    

Next, we will update the weights of networks $Q$ and $\hat Q$ using a custom training loop. Therefore we need to retrieve the gradients via a `GradientTape` instance, and then call `optimizer.apply_gradients()` to update our Q-network.  The `@tf.function` decorator can increase performance and shorten training time. 

In [40]:
# Line 12 - 14 of the algorithm

@tf.function # This is just for faster training
def agent_learn(experiences, gamma):

    with tf.GradientTape() as tape:
        loss = compute_loss(experiences, gamma, q_network, target_q_network)
    
    # Autodifferentiation: find gradients of loss with respect to weights
    gradients = tape.gradient(loss, q_network.trainable_variables)

    # Apply the gradients to the optimizer to update the weights of q_network
    optimizer.apply_gradients(zip(gradients, q_network.trainable_variables))

    # update the weights of target q_network using soft update from q network
    update_target_network(TAU)

def update_target_network(tau):
    for target_weights, q_net_weights in zip(target_q_network.weights, q_network.weights):
        target_weights.assign(tau * q_net_weights + (1-tau) * target_weights)

### Train the Agent (Putting it all together)
1. Initialize memory buffer with capacity N = MEMORY_SIZE using a deque as data structure

2. Initialize q_network (done)

3. Initialize target_q_network by setting its weights equal to q_network (done)

4. Start outer loop: M = num_episodes = 2000 in which the agent should solve the environment

5. Use .reset() method to reset environment to initial state (for each new episode)

6. Star inner loop: T = max_num_timesteps = 1000

7. Agent observes current state and choose action using epsilon-greedy policy, set epislon initially as 1 (random) and gradually decrease (decay rate) to a min of 0.01 -- `get_action()`

8. Use the .step() method to take the given action in the environment and get reward and next_state

9. Store experience(state, action, reward, next_state, done) to memory_buffer

10. Check if conditions met to perform a learning udpate (so not too frequently, in `check_update_conditions`)
    
    a. C = NUM_STEPS_FOR_UPDATE = 4 time steps have occured
    
    b. Memory_buffer has enough experience tuples to fill a mini-batch sample

11. If update is True, perform learning update: sample mini-batch randomly, set y targets and calculate loss, perform gradient descent, update weights of the networks -- `agent_learn()` (done)

15. End of iteration of inner loop: 
    
    a. Set next_state as our new state so the loop can start again from this new state
    
    b. Check if episodes has reached a terminal state (done = True) -> break out of inner loop

16. End of outer loop iteration: 
    
    a. Update value of epsilon 
    
    b. Check if environment has been solved -- agent receives avg 200 points in the last 100 episodes
    
    c. If not solved, continue outer loop and start a new episode

#### Other helper functions

In [31]:
def get_action(q_values, epsilon):
    if random.random() < epsilon: 
        return random.randint(0, num_actions - 1)
    else:
        return np.argmax(q_values)
    
def check_update_conditions(t, num_steps_for_update, memory):
    if t % num_steps_for_update == 0 and len(memory) >= MINIBATCH_SIZE:
        return True
    else: 
        return False

def get_experiences(memory):
    experiences = random.sample(memory, MINIBATCH_SIZE)
    
    states = tf.convert_to_tensor(np.array([e.state for e in experiences if e is not None]),dtype=tf.float32)
    actions = tf.convert_to_tensor(np.array([e.action for e in experiences if e is not None]),dtype=tf.float32)
    rewards = tf.convert_to_tensor(np.array([e.reward for e in experiences if e is not None]),dtype=tf.float32)
    next_states = tf.convert_to_tensor(np.array([e.next_state for e in experiences if e is not None]),dtype=tf.float32)
    dones = tf.convert_to_tensor(np.array([e.done for e in experiences if e is not None]).astype(np.uint8),dtype=tf.float32)

    return (states, actions, rewards, next_states, dones)

def get_new_eps(epsilon):
    return max(E_MIN, epsilon * E_DECAY)

### Main Execution

In [41]:
# Putting it all together:
start = time.time()

num_episodes = 2000
max_num_timesteps = 1000

num_p_av = 100 # Number of episodes to average over, we need last 100 episodes to be > 200
epsilon = 1.0

total_point_history = []

memory_buffer = deque(maxlen=MEMORY_SIZE)

for i in range(num_episodes):
    state, _ = env.reset()
    total_points = 0

    for t in range(max_num_timesteps):

        # From current state S choose an action A using epsilon-greedy policy
        state_qn = np.expand_dims(state, axis=0) # reshape state
        q_values = q_network(state_qn, training=False)
        action = get_action(q_values, epsilon)

        # Take action A and receive reward R and the next state S'
        next_state, reward, done, _, _ = env.step(action)

        memory_buffer.append(experience(state, action, reward, next_state, done))

        update = check_update_conditions(t, NUM_STEPS_FOR_UPDATE, memory_buffer)

        if update:
            # random sample mini-batch
            experiences = get_experiences(memory_buffer)
            agent_learn(experiences, GAMMA)

        state = next_state.copy()
        total_points += reward

        if done: break
    
    total_point_history.append(total_points)
    av_latest_points = np.mean(total_point_history[-num_p_av:]) # Last 100 episodes

    epsilon = get_new_eps(epsilon)

    if (i+1) % num_p_av == 0:
        print(f"\rEpisode {i+1} | Total point average of the last {num_p_av} episodes: {av_latest_points:.2f}")
    
    # if last 100 episodes avg >= 200 points, we can end
    if av_latest_points >= 200:
        print(f"\nEnvironment solved in {i+1} episodes!")
        q_network.save("lunar_lander_model.keras")
        break

tot_time = time.time() - start
print(f"\nTotal Runtime: {tot_time:.2f} s ({(tot_time/60):.2f} min)")

2024-08-01 21:13:32.808500: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Episode 100 | Total point average of the last 100 episodes: -150.47
Episode 200 | Total point average of the last 100 episodes: -92.40
Episode 300 | Total point average of the last 100 episodes: -19.58
Episode 400 | Total point average of the last 100 episodes: 30.40
Episode 500 | Total point average of the last 100 episodes: 136.06
Episode 600 | Total point average of the last 100 episodes: 181.03
Episode 700 | Total point average of the last 100 episodes: 197.47

Environment solved in 702 episodes!

Total Runtime: 1506.01 s (25.10 min)
